[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/modelagem_4.ipynb)

# Modelagem e Gestão de Processos


**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**



In [ ]:
import simpy
import random
import numpy as np

In [ ]:
random.seed(2)

### Exemplo de processo que não alcança estado estável

Exemplo de processo M/M/2, com $\lambda = 1$ cliente/minuto e $\mu = 0,2$ clientes/minuto.

Como $\rho = \frac{\lambda}{s\times \mu} = \frac{1}{0,4} >1$, é esperado que o processo não estabilize.

A fila explodirá. 

Altere o código e calcule o tempo médio de espera dos clientes que finalizaram o serviço.

In [ ]:
def estabelecimento (environment, nome_cliente, servidor):
    #imprima na tela o tempo de chegada.
    print (f"{nome_cliente} chega ao estabelecimento em {environment.now}") 
    
    #guarde a informação na variável tempo_chegada.
    tempo_chegada = environment.now 
    
    # A atividade requer um servidor. Usamos o with para demonstrar isso. 
    with servidor.request() as req:
        # Espere até o servidor estar disponível e guarde o tempo.
        yield req 
        
        # Escreva na tela o tempo de início do serviço
        print (f"{nome_cliente} inicia o serviço em {environment.now}") 
        # guarde o tempo de início em uma variável chamada tempo_inicio
        tempo_inicio = environment.now

        # tempo de espera será o tempo_inicio - tempo_chegada 
        tempo_em_fila.append(tempo_inicio - tempo_chegada)

        # O tempo estimado segue uma exponencial trabalha com Mu=0,2
        yield environment.timeout(random.expovariate(0.2))  
        
        #Imprima na tela o tempo de saída do sistema
        print (f"{nome_cliente} finaliza o serviço e sai do estabelecimento em {environment.now}")
        tempo_saida = environment.now
        
        # O tempo no sistema será tempo_saida - tempo_chegada
        tempo_no_sistema.append(tempo_saida - tempo_chegada) 

def chegadas (environment):
    id = 1 #guarda o id do cliente 
    
    # Enquanto houver simulação:
    while True:
        # Passa um tempo até a próxima chegada, seguindo exponencial com média (1/lambda)
        yield environment.timeout(random.expovariate(1))
        
        # Um cliente chega no processo
        environment.process(estabelecimento (environment, 'Cliente %d' % id, servidor))
        
        # O próximo cliente terá id = id + 1
        id += 1



In [ ]:
tempo_em_fila = [] # uma lista vazia que receberá os tempos em fila
tempo_no_sistema = [] # uma lista vazia que receberá os tempos no sistema de filas

ambiente = simpy.Environment()
servidor = simpy.Resource(ambiente, capacity=2)
ambiente.process(chegadas(ambiente))
ambiente.run(until = 480)

In [ ]:
print (f"Número de clientes que iniciaram o atendimento : {len(tempo_em_fila)}")
print (f"O tempo de espera médio: {np.mean(tempo_em_fila).round(2)} minutos")
print (f"O tempo médio no sistema: {np.mean(tempo_no_sistema).round(2)} minutos")

Veja que o tempo de espera médio foi de mais de 2 horas.

Na verdade, só contabilizamos tempos de espera que conseguiram iniciar o atendimento.

Perceba que chegaram mais de 400 clientes e menos da metade deles foram atendidos. 